In [124]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import astropy
import extinction
import sncosmo
from astropy.table import Table

In [125]:
'''load in file with wavelength separations'''
df = pd.read_csv('./d_lambda.txt', comment='#', sep = '\s+')

In [127]:
'''define the six SPHEREx bands using specific wavelengths separated 
    by distances as given in the SPHEREx documentation from file above'''
wl1 = [] #angstrom, first wavelength value of SPHEREx band 1
wl2 = []
wl3 = []
wl4mod = []
wl4 = []
wl5 = []
wl6 = []

for i in range(len(df['lambda'])):
    new_wl = (df['lambda'][i]*10000) #convert to angstroms, sncosmo pref.
    #print(new_wl)
    if new_wl <= 11200:
        wl1.append(new_wl)
    if (new_wl >= 11000) & (new_wl <= 16500):
        wl2.append(new_wl)
    if (new_wl >= 16300) & (new_wl <= 24400):
        wl3.append(new_wl)
    if (new_wl >= 24000) & (new_wl <= 33897.8):
        wl4mod.append(new_wl)
    if (new_wl >= 24000) & (new_wl <= 38500):
        wl4.append(new_wl)
    if (new_wl >= 38100) & (new_wl <= 44300):
        wl5.append(new_wl)
    if (new_wl >= 44100) & (new_wl <= 50100):
        wl6.append(new_wl)

12

In [129]:
'''further explaned in documentation, but transmission should be 
    irrelevant, set to 1 for all cases'''
T_len12 = np.ones(12) #for wavelength arrays w 14 entries
T_len16 = np.ones(16) #for wavelength arrays w 16 entries
T_len17 = np.ones(17) #for wavelength arrays w 17 entries

In [130]:
'''make sncosmo Bandpasses'''
band1 = sncosmo.Bandpass(wl1, T_len16)
sncosmo.register(band1, 'band1', force=True)

band2 = sncosmo.Bandpass(wl2, T_len16, name='band2')
sncosmo.register(band2, 'band2', force=True)

band3 = sncosmo.Bandpass(wl3, T_len17, name='band3')
sncosmo.register(band3, 'band3', force=True)

band4mod = sncosmo.Bandpass(wl4mod, T_len12, name='band4mod')
sncosmo.register(band4mod, 'band4mod', force=True)

band4 = sncosmo.Bandpass(wl4, T_len17, name='band4')
sncosmo.register(band4, 'band4', force=True)

band5 = sncosmo.Bandpass(wl5, T_len16, name='band5')
sncosmo.register(band5, 'band5', force=True)

band6 = sncosmo.Bandpass(wl6, T_len17, name='band6')
sncosmo.register(band6, 'band6', force=True);

In [131]:
'''Now we simulate some SNe'''

'Now we simulate some SNe'

In [132]:
'''load in file with SN params'''
FITRES = pd.read_csv('./FITOPT000.FITRES', sep = '\s+', comment = '#')

In [133]:
'''SN 98342'''
model = sncosmo.Model('salt3-nir')
'''the SALT3 model runs on 5 params: z, t0, x0, x1, and c which we can 
    set based on the SN of interest'''
for i in range(len(FITRES.CID)):
    if FITRES.CID[i] == 98342:
        model.parameters[0] = FITRES.zHD[i]
        model.parameters[1] = FITRES.PKMJD[i]
        model.parameters[2] = FITRES.x0[i]
        model.parameters[3] = FITRES.x1[i]
        model.parameters[4] = FITRES.c[i]
        params = {'z': FITRES.zHD[i], 't0': FITRES.PKMJD[i], 'x0': FITRES.x0[i], 'x1': FITRES.x1[i], 'c': FITRES.c[i]}

In [134]:
'''load in file with observables'''
LCPLOT = pd.read_csv('./LSSTDDF_EUCLID.LCPLOT.TEXT', sep = ', ', comment = '#')

C:\Users\avacb\AppData\Local\Temp\ipykernel_16720\3826675009.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  LCPLOT = pd.read_csv('./LSSTDDF_EUCLID.LCPLOT.TEXT', sep = ', ', comment = '#')


In [64]:
'''based on magnitude similarity, 
    I use noise and time from LSST or Euclid:
    band1 = LSST y, band2 = Euclid J, band3-band6 = Euclid H'''
times = []
skynoise = []

for i in range(len(LCPLOT.CID)):
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'y') & (LCPLOT.MJD[i] == 62176.961):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'J') & (LCPLOT.MJD[i] == 62176.961):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'H') & (LCPLOT.MJD[i] == 62176.961):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'H') & (LCPLOT.MJD[i] == 62176.961):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'H') & (LCPLOT.MJD[i] == 62176.961):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'H') & (LCPLOT.MJD[i] == 62176.961):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])'''

obs = Table({'time': times,
             'band': ['band1', 'band2', 'band3', 'band4'],
             'gain': [1., 1., 1., 1.],
             'skynoise': skynoise,
             'zp': [21., 21., 21., 21.], 
             'zpsys': ['ab', 'ab', 'ab', 'ab']})
                    
lcs = sncosmo.realize_lcs(obs, model, [params])

ValueError: bandpass 'band4' [24115.2, .., 37981] outside spectral range [3389.78, .., 15593]

In [38]:
model.parameters[1]

63213.0

In [135]:
times = []
band = []
skynoise = []
zpsys = []

for i in range(len(LCPLOT.CID)):
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'y'):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
        band.append('band1')
        zpsys.append('ab')
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'J'):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
        band.append('band2')
        zpsys.append('ab')
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'H'):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
        band.append('band3')
        zpsys.append('ab')
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'H'):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
        band.append('band4mod')
        zpsys.append('ab')
    '''if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'H'):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
        band.append('band5')
        zpsys.append('ab')
    if (LCPLOT.CID[i] == 98342) & (LCPLOT.BAND[i] == 'H'):
        skynoise.append(LCPLOT.FLUXCAL_ERR[i])
        times.append(LCPLOT.MJD[i])
        band.append('band6')
        zpsys.append('ab')'''

gain = np.ones(len(times))
zp = 21.*np.ones(len(times))

obs = Table({'time': times,
             'band': band,
             'gain': gain,
             'skynoise': skynoise,
             'zp': zp, 
             'zpsys': zpsys})
                    
lcs = sncosmo.realize_lcs(obs, model, [params])
print(lcs[0])

   time     band            flux         fluxerr  zp  zpsys
--------- -------- --------------------- ------- ---- -----
62070.355    band1     8.939833310548382  21.727 21.0    ab
62075.219    band1    27.906656476759498  21.905 21.0    ab
62099.125    band1 -0.009035125750976154  26.943 21.0    ab
62101.211    band1   -15.050011255757111  26.022 21.0    ab
62101.305    band1    10.862071904520974   30.86 21.0    ab
62103.199    band1    -32.53422785951116   20.87 21.0    ab
62125.039    band1    15.796443283760327  19.553 21.0    ab
62129.316    band1    -51.99142732517223   72.69 21.0    ab
62133.172    band1    13.985659977568226  15.497 21.0    ab
62159.168    band1    -36.79925681858679  34.358 21.0    ab
      ...      ...                   ...     ...  ...   ...
62166.961 band4mod   0.23326256401278803  2.2067 21.0    ab
62168.961    band3     0.967369257798615  2.1293 21.0    ab
62168.961 band4mod   -0.6453071518219454  2.1293 21.0    ab
62170.961    band3    2.6172397832859535

In [136]:
result, fitted_model = sncosmo.fit_lc(lcs[0], model, ['z', 't0', 'x0', 'x1', 'c'], bounds={'z':(0.3, 0.7)})  # bounds on parameters (if any)

C:\Users\avacb\anaconda3\lib\site-packages\sncosmo\fitting.py:187: RuntimeWarning: Dropping following bands from data: band4mod(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
C:\Users\avacb\anaconda3\lib\site-packages\sncosmo\fitting.py:235: RuntimeWarning: invalid value encountered in true_divide
  significant_data = data[(data.flux / data.fluxerr) > minsnr]


DataQualityError: No data points with S/N > 5.0. Initial guessing failed.

In [101]:
'''Pivot to LSST'''

'Pivot to LSST'